In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)


23/04/23 18:28:31 WARN Utils: Your hostname, jakub-G3-3590 resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp3s0)
23/04/23 18:28:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/23 18:28:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
brzydki_json_path = "../../Lab_7/brzydki.json"

In [25]:
df = spark.read.format("json") \
    .option("multiline", "true") \
    .load(brzydki_json_path) \
    .selectExpr(
    "explode(features) as feature",
    "jobDetails",
    "maiaExtractProcessDetails",
)

In [26]:
df.show()

+--------------------+--------------------+-------------------------+
|             feature|          jobDetails|maiaExtractProcessDetails|
+--------------------+--------------------+-------------------------+
|{{[[[445751.0,244...|{2022-08-06T03:29...|     {Incremental Upda...|
|{{[[446492.041, 2...|{2022-08-06T03:29...|     {Incremental Upda...|
+--------------------+--------------------+-------------------------+



In [38]:
df.selectExpr("jobDetails").printSchema()

root
 |-- jobDetails: struct (nullable = true)
 |    |-- changesTimestamp: string (nullable = true)
 |    |-- jobId: long (nullable = true)
 |    |-- jobName: string (nullable = true)
 |    |-- jobType: string (nullable = true)
 |    |-- workspaceId: long (nullable = true)



In [37]:
df_job_details = df.selectExpr(
    "jobDetails.changestimestamp",
    "jobDetails.jobId",
    "jobDetails.jobName",
    "jobDetails.jobType",
    "jobDetails.workspaceId"
)
df_job_details.show()

+--------------------+------+--------------------+-------+-----------+
|    changestimestamp| jobId|             jobName|jobType|workspaceId|
+--------------------+------+--------------------+-------+-----------+
|2022-08-06T03:29:46Z|345345|XSDR_344985_WGZX_...| Stereo|     345353|
|2022-08-06T03:29:46Z|345345|XSDR_344985_WGZX_...| Stereo|     345353|
+--------------------+------+--------------------+-------+-----------+



In [40]:
df.selectExpr("maiaExtractProcessDetails").printSchema()

root
 |-- maiaExtractProcessDetails: struct (nullable = true)
 |    |-- extractMode: string (nullable = true)
 |    |-- extractionTimestamp: string (nullable = true)
 |    |-- extractorVersion: string (nullable = true)
 |    |-- fileSequenceNumber: long (nullable = true)
 |    |-- maiaSavepoint: string (nullable = true)
 |    |-- queryBox: string (nullable = true)



In [42]:
df_maia = df.selectExpr(
    "maiaExtractProcessDetails.extractMode",
    "maiaExtractProcessDetails.extractionTimestamp",
    "maiaExtractProcessDetails.extractorVersion",
    "maiaExtractProcessDetails.fileSequenceNumber",
    "maiaExtractProcessDetails.maiaSavepoint",
    "maiaExtractProcessDetails.queryBox",
)
df_maia.show()

+------------------+--------------------+----------------+------------------+-------------+--------+
|       extractMode| extractionTimestamp|extractorVersion|fileSequenceNumber|maiaSavepoint|queryBox|
+------------------+--------------------+----------------+------------------+-------------+--------+
|Incremental Update|2022-08-06T03:30:35Z|            1.10|            110572|         null|    null|
|Incremental Update|2022-08-06T03:30:35Z|            1.10|            110572|         null|    null|
+------------------+--------------------+----------------+------------------+-------------+--------+



In [43]:
df.selectExpr("feature.geometry").printSchema()

root
 |-- geometry: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |-- type: string (nullable = true)



In [56]:
df_feature_geometry = df.selectExpr(
    "feature.geometry.coordinates",
    "feature.geometry.type"
)
df_feature_geometry.show()

+--------------------+----------+
|         coordinates|      type|
+--------------------+----------+
|[[[445751.0,24497...|   Polygon|
|[[446492.041, 245...|LineString|
+--------------------+----------+



In [57]:
df.selectExpr("feature.properties").printSchema()

root
 |-- properties: struct (nullable = true)
 |    |-- accessTopologyComponent: string (nullable = true)
 |    |-- administrativeExceptionComponent: string (nullable = true)
 |    |-- administrativeUnitComponent: string (nullable = true)
 |    |-- alternativeToids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- anomalyComponent: string (nullable = true)
 |    |-- areaHeightComponent: string (nullable = true)
 |    |-- baseFormComponent: struct (nullable = true)
 |    |    |-- applicableContracts: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- componentId: string (nullable = true)
 |    |    |-- form: string (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- versionCreationMetadata: struct (nullable = true)
 |    |    |    |-- captureMethod: string (nullable = true)
 |    |    |    |-- confidenceLevel: string (nullable = true)
 |    |    |    |-- editDate: string (nu

In [58]:
df_properties = df.selectExpr(
    "feature.properties.accessTopologyComponent",
    "feature.properties.administrativeExceptionComponent",
    "feature.properties.administrativeUnitComponent",
    "feature.properties.alternativeToids",
)
df_properties.show()

+-----------------------+--------------------------------+---------------------------+----------------+
|accessTopologyComponent|administrativeExceptionComponent|administrativeUnitComponent|alternativeToids|
+-----------------------+--------------------------------+---------------------------+----------------+
|                   null|                            null|                       null|              []|
|                   null|                            null|                       null|              []|
+-----------------------+--------------------------------+---------------------------+----------------+



In [67]:
df_base_form_component = df.selectExpr(
    "feature.properties.baseFormComponent.versionCreationMetaData.*"
)
df_base_form_component.show()

+-------------+---------------+----------+------------+------------+---------------+
|captureMethod|confidenceLevel|  editDate|evidenceDate|jobReference|reasonForChange|
+-------------+---------------+----------+------------+------------+---------------+
|      Unknown|      Confident|2005-09-20|  2005-09-20|           0|     Refinement|
|   Collection|      Confident|2018-07-26|  2018-07-26|    42334591|           Real|
+-------------+---------------+----------+------------+------------+---------------+

